In [ ]:
import pandas as pd
import time
import requests
import json

In [4]:
def get_gender(name):
    last,first,*_ = name.split(" ::: ")
    path = "http://abel.lis.illinois.edu/cgi-bin/ethnea/search.py?Fname={0}&Lname={1}&format=json".format(first, last)
    r = requests.get(path)
    text = r.text.strip().replace('\'', '"')
    x = json.loads(text)
    gender,ethnic = x['Genni'],x["Ethnea"]
    
    #for testing
    #print("{0} {1}".format(first,last), end=" --> ")
    #print("empirical: {0}".format(empirical))
    
    return gender,ethnic

In [5]:
def gender_main(names,fname):
    i = 0
    with open(fname,"a+") as f:
        for name in names:
            i += 1
            if ((i%1000) == 0):
                print("{:2.2f}".format(i/len(names)*100),end = "-")
                
            try:
                gen,eth = get_gender(name)
            except:
                print(name)
            f.write("{}\t{}\t{}\n".format(name,gen,eth))

                


In [6]:
authors = pd.read_csv("./data/authors.tsv",sep="\t")
authors = authors.loc[authors["corr"]==1]

In [7]:
#read editor gender
editors = set(pd.read_csv("data/editors_gender.tsv",sep="\t")["name"])
#read gender_corr
ethnicity = set(pd.read_csv("data/gender_corr.tsv",sep="\t",header=None,names=["name","gender","eth"])["name"])
#Get rest of names
already_names = editors | ethnicity

unique_names = list(set(authors["name"]) - already_names)

In [ ]:
gender_main(unique_names,"./data/gender_corr.tsv")